## INITIALIZE REQUIREMENTS

### Load Dependencies

In [1]:
%matplotlib inline
# !pip install python-dotenv
# load_dotenv()
# from dotenv import load_dotenv

import os, glob, warnings, datacube, rasterio, folium, json, io, statistics
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as rio
import pandas as pd
from PIL import Image
from fpdf import FPDF
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import Point
from shapely.geometry import Polygon
from datetime import date
from dateutil.relativedelta import relativedelta


from scipy.ndimage import uniform_filter
from scipy.ndimage import variance
from skimage.filters import threshold_minimum
from datacube.utils.geometry import Geometry

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.areaofinterest import define_area

from typing import Literal

from IPython.display import clear_output
from IPython.display import display

warnings.filterwarnings("ignore")

### Connect to the datacube

Connect to the datacube so we can access DEA data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [2]:
dc = datacube.Datacube(app="Radar_water_detection")

### Timerange Definitions

In [3]:
ERR_FOLDER_ID = None
# Define main time period of analysis
timerange = ("2023-11", "2024-11")

In [4]:
threshold_aoi = -21

### Filter and Classifier Functions

In [5]:
# Function to apply lee filtering on S1 image. Speckle Filter
def lee_filter(da, size):
    """
    Apply lee filter of specified window size.
    Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python

    """
    img = da.values
    img_mean = uniform_filter(img, size)
    img_sqr_mean = uniform_filter(img**2, size)
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)

    return img_output


# Classifier Function
def S1_water_classifier(da, threshold):
    water_data_array = da < threshold
    return water_data_array.to_dataset(name="s1_water")

### Operational Funtions

In [6]:
def get_months(timerange: tuple) -> list[date]:
    """
    Converts the timerange into a list of months.

    Parameters:
    timerange ("YYYY-MM", "YYYY-MM"): tuple, required
        Timerange of the analysis. Start month to end month.

    Returns:
    months: list[date]
    """
    syear, smonth = timerange[0].split("-")
    syear = int(syear)
    smonth = int(smonth)
    start = date(syear, smonth, 30)

    eyear, emonth = timerange[1].split("-")
    eyear = int(eyear)
    emonth = int(emonth)
    end = date(eyear, emonth, 30)
    months = []
    while start <= end:
        months.append(start.strftime("%Y-%m"))
        start = start + relativedelta(months=+1)

    return months

In [7]:
def gen_elog(e_log: list) -> list:
    """
    Writes a the error log json file and uploads it to the google drive folder ID, if specified.

    Parameters:
    e_log: list, required
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.

    Returns:
    e_log: list
        Error log list having the same values as the input parameter
    """
    e_log = np.array(e_log)
    with open("error_centroids.json", "w") as filehandle:
        json.dump(e_log.tolist(), filehandle)

    # read error log from disk
    with open("error_centroids.json") as f:
        e_log = json.load(f)
    for e in e_log:
        e[0] = float(e[0])
        e[1] = float(e[1])
        e[2] = int(e[2])

    if ERR_FOLDER_ID != None:
        try:
            gd.upload_files(["error_centroids.json"], ERR_FOLDER_ID, False)
        except Exception as e:
            print("FAILED TO UPLOAD ERROR LOG FILE REASON:{}".format(e))
    else:
        print("Error Log json created and stored on disc")

    return e_log

In [8]:
def gen_table():
    # Init pd.DataFrame
    headers = ["cell_id", "total", "lat", "lon", "datasets"]
    months = get_months(timerange)
    headers.extend(months)
    count_table = pd.DataFrame(columns=headers)
    return count_table

In [9]:
# Iterate through the input grid


def iterate_grid(aoi_m: list, c: list, count_table) -> list:
    """
    Iterates through every feature (cell) in the AOI grid.

    Parameters:
    aoi_m: list, required
        List of geojson feature collection (cells) that make up the entire grid.
    c: list, required
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails

    Returns:
    e_log: list
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.
    """
    e_log = []
    cell = 1
    months = months = get_months(timerange)

    # Run the main iterator
    for aoi, i in zip(aoi_m, c):
        geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
        geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)
        g = geopolygon_gdf.centroid

        print(
            "\n\n"
            + "\033[32m"
            + "PROCESSING GRID CELL ID {} NO. {}/{} CENTROID ({}, {})".format(
                i[2], cell, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
            )
            + "\033[0m"
        )

        # Get the latitude and longitude range of the geopolygon
        lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
        lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])

        # Load Sentinel1 data
        try:
            S1 = load_ard(
                dc=dc,
                products=["s1_rtc"],
                measurements=["vh"],
                y=lat_range,
                x=lon_range,
                time=timerange,
                output_crs="EPSG:6933",
                resolution=(-20, 20),
                group_by="solar_day",
                dtype="native",
            )
        except Exception as e:
            # Log error aoi centroids and keep looping
            e_log.append([g.x[0], g.y[0], i[2], "P"])

            print(
                "\n\n"
                + "\033[31m"
                + "ERROR PROCESSING GRID CELL {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(
                    i[2], len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
                )
                + "\033[0m"
            )

            print("PROCESS ERROR: {}".format(e))
            cell += 1
            continue

        datasets = S1.time
        dn = len(datasets.time)

        # Initialize row for the count_table
        count_row = [i[2], len(aoi_m), round(g.y[0], 5), round(g.x[0], 5), dn]

        # The lee filter above doesn't handle null values
        # We therefore set null values to 0 before applying the filter
        valid = np.isfinite(S1)
        S1 = S1.where(valid, 0)

        # Create a new entry in dataset corresponding to filtered VV and VH data
        S1["filtered_vh"] = S1.vh.groupby("time").apply(lee_filter, size=7)

        # Null pixels should remain null
        S1["filtered_vh"] = S1.filtered_vh.where(valid.vh)

        # Convert the digital numbers to dB
        S1["filtered_vh"] = 10 * np.log10(S1.filtered_vh)

        threshold_vh = threshold_aoi

        S1["water"] = S1_water_classifier(S1.filtered_vh, threshold_vh).s1_water
        S1Water = S1.water
        S1_BIN = S1Water.where(S1Water > 0)

        print("Generating row data...")
        for month in months:
            S1_month = S1_BIN.sel(time=[month], method="nearest").mean(dim="time")
            # Get count of all pixels that are water and not np.nan
            count = np.count_nonzero(~np.isnan(S1_month.values))
            count_row.append(count)

        print(count_row)

        print("Adding to table")
        count_table = pd.concat(
            [pd.DataFrame([count_row], columns=count_table.columns), count_table],
            ignore_index=True,
        )

        count_table.to_csv("output/csv/cell_counts.csv", index=False, mode="w")

        cell += 1

    if len(e_log) == 0:
        print("\n\n" + "\033[32m" + "GRID PROCESSED SUCCESSFULLY" + "\033[0m" + "\n\n")

    e_log = gen_elog(e_log)

    # return e_log to be run again
    return e_log, count_table

In [10]:
# Crete the aoi-mosaic - aoi_m
def gen_aoim(c: list, b: float, count_table) -> list:
    """
    Generates the feature collection list (list of cells) using centroid coordinates and a buffer distance. Calls the main iterator for execution as well.

    Parameters:
    c: list, required
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails.
    b: float, required
        Cell half-dimension in degrees (EPSG:4326). Creates a cell by adding this distance to the centroid coordinates.

    Returns:
    e_log: list
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.
    """
    aoi_m = []
    for i in c:
        aoi_m.append(define_area(i[1], i[0], buffer=b))
    # print(c, len(aoi_m))
    e_log, count_table = iterate_grid(aoi_m, c, count_table)

    # return e_log to be run again
    return e_log, count_table

In [11]:
# Visualize input file
def view_input(gdf_list: list[gpd.GeoDataFrame], grid_c: list) -> None:
    """
    Visualizes cells and respective IDs  on a basemap.

    Parameters:
    gdf_list:list[gpd.GeoDataFrame], required
        List of geodataframes to be visualized.
    grid_c:list, required
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails.

    Returns:
    None

    """
    print("Visualizing data...")

    # Dissolve
    p = gdf_list[0].dissolve()
    center = p.centroid
    map = folium.Map(location=[center.y, center.x], tiles="CartoDB Positron")

    for gdf in gdf_list:
        folium.GeoJson(gdf, name="{}".format(gdf)).add_to(map)

    for c in grid_c:
        folium.Marker(
            location=[c[1], c[0]],
            popup=f"Centroid: {c[1]}, {c[0]}",
            icon=folium.DivIcon(
                icon_size=(10, 10),
                icon_anchor=(0, 0),
                html='<div style="font-size: 10pt">{}</div>'.format(c[2]),
            ),
        ).add_to(map)

    bounds = gdf_list[0].total_bounds.tolist()
    map.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])
    display(map)

In [12]:
# Create grid
# from shapely import intersection as intersect
def create_grid(adm0: gpd.GeoDataFrame, size: float) -> gpd.GeoDataFrame:
    """
    Divides adm0 AOI vectorfile into square grid based on size

    Parameters:
    adm0:gpd.GeoDataFrame, required
        AMD0 GeoDataFrame created from ADM0 input vector file
    size:float, required
        Grid cell size in degrees (EPSG:4326)

    Returns:
    grid: gpd.GeoDataFrame
        The generated grid GeoDataFrame
    """
    bounds = adm0.bounds
    minx = bounds.minx[0]  # only 1 feature at the 0th index
    miny = bounds.miny[0]
    maxx = bounds.maxx[0]
    maxy = bounds.maxy[0]

    grid = gpd.GeoDataFrame()
    for x0 in np.arange(minx, maxx, size):
        for y0 in np.arange(miny, maxy, size):
            x1 = x0 + size
            y1 = y0 + size
            d = {"geometry": [Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])]}
            cell = gpd.GeoDataFrame(d, crs="EPSG:4326")
            flag = adm0.intersection(cell)
            if flag[0].is_empty == False:
                grid = pd.concat([grid, cell])

    return grid

In [13]:
# Check CRS and convert to 4326 if required
def crs_check(shp: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Checks input GeoDataFrame CRS and converts to EPSG 4326, if different.

    Parameters:
    shp: gpd.GeoDataFrame, required
        Input GeoDataFrame to check.

    Returns:
    shp: gpd.GeoDataFrame
        As is or converted GeoDataFrame.
    """
    if shp.crs != "EPSG:4326":
        print("Added ADM0 CRS is {}. Converting to EPSG:4326...".format(shp.crs))
        shp = shp.to_crs("EPSG:4326")
        if shp.crs == "EPSG:4326":
            print("Done")

    return shp

In [14]:
def check_inp(x: str) -> None:
    """
    Checks if input is "y" or "n"

    Parameters:
    x: str, required
        String input to be checked

    Returns:
    None
    """
    if x not in ["y", "n"]:
        raise ValueError("Invalid input, must be 'y' or 'n'")
    elif x == "n":
        raise RuntimeError(
            "Excecution terminated. Make necessary changes before running again"
        )


def exec_checks(count_table) -> None:
    """
    Performs checks and Run the entire application

    Parameters:
    None

    Returns:
    None
    """

    inst = """
Before running the execution, ensure all requirements have been met:
1. Check input shapefile
3. Check grid size
    
    """
    print(inst)

    x = input("Input shapefile/geojson verified? (y/n):")
    check_inp(x)

    input("Grid (size) verified? (y/n):")
    check_inp(x)

    z = input("\nBegin threshold calculation for input shapefile/geojson? (y/n):")
    if z not in ["y", "n"]:
        raise ValueError("Invalid input, must be 'y' or 'n'")
    elif z == "n":
        raise RuntimeError(
            "Excecution terminated. Make necessary changes before running again"
        )
    elif z == "y":
        print("Starting execution...")
        # get e_log with centroids, cell_id and error message
        # Calling gen_aoim will run the entire Application
        e_log, count_table = gen_aoim(c, buffer, count_table)
        return e_log, count_table

In [15]:
def del_files(path: str, ext: str) -> None:
    """
    Deletes all files with specified extention at specified folder path

    Parameters:
    path:str, required
        Folder path.
    ext:str, required
        File extension. "*" for all files.

    Returns:
    None
    """
    res_files = False
    loc = os.path.join(path, ext)
    files = glob.glob(loc)
    if len(files) > 0:
        res_files = True
        print("Found {} files. Deleting...".format(len(files)))
        for f in files:
            os.remove(f)

    return res_files

In [16]:
def clean_dirs() -> None:
    """
    Creates directories if they dont exist or deletes residual files if they exist.

    Parameters:
    None

    Returns:
    None
    """
    dirs_exist = False
    dir_dict = {
        "sd_flood": "output/flood",
        "sd_preflood": "output/preflood",
        "sd_flood_extents": "output/flood_extents",
    }

    for k in dir_dict:
        if not os.path.exists(dir_dict[k]):
            os.makedirs(dir_dict[k])
        else:
            dirs_exist = True
            r = del_files(dir_dict[k], "*")

    if dirs_exist:
        print("Output folders alredy exist.")
    else:
        print("Output folders created.")

    if not r:
        print("No residual files to delete.")

### Upload Gridded Vector File

In [17]:
# Load file from sandbox disc. file should be present in 'input' folder
# grid = gpd.read_file("input/Lake Chad.geojson")
# grid = gpd.read_file("input/TCD_55KM_4CTEST.geojson")
# grid = gpd.read_file("input/TCD_55KM_BASE.geojson")
# grid = gpd.read_file("input/TCD_55KM_ERR.geojson")

# Create and clean sandbox output directories
clean_dirs()

# shp = "input/LC_9C_TH.geojson"
shp = "input/TCD_55KM_BASE.geojson"
adm0_base = gpd.read_file(shp)  # adm0 base
adm0_base = adm0_base.dissolve()
adm0 = adm0_base.buffer(0.2)  # adm0 with 20KM boundary buffer
adm0 = crs_check(adm0)

size = 0.5  # Grid cell size 0.5 ~ 55KM
buffer = size / 2  # cell buffer around the centroid to create the cell

grid = create_grid(adm0, size)

# Calculate centroids and store in centroid list c[].
c = []
g = grid.centroid

cell_id = 1
for i in g:
    c.append(
        [round(i.x, 5), round(i.y, 5), cell_id, None]
    )  # The array c[] has four values: x, y, cell_id and None. None will store the "P" or "U" error value
    cell_id += 1

# Splice c to test with a few cells
# c = c[:2]  # 0-9 cells

view_input([grid, adm0], c)

Output folders alredy exist.
No residual files to delete.
Visualizing data...


### Check and Run Application

In [18]:
count_table = gen_table()

In [ ]:
# Calls the checklist function
e_log, count_table = exec_checks(count_table)


Before running the execution, ensure all requirements have been met:
1. Check input shapefile
3. Check grid size
    
    


Input shapefile/geojson verified? (y/n): y
Grid (size) verified? (y/n): y

Begin threshold calculation for input shapefile/geojson? (y/n): y


Starting execution...


PROCESSING GRID CELL ID 1 NO. 1/522 CENTROID (12.99107, 13.52348)
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 61 time steps
Generating row data...
[1, 522, 12.99107, 13.52348, 61, 6124766, 6250540, 6157840, 6214693, 6284937, 6106042, 6403297, 6256355, 5653617, 361449, 28281, 104684, 3157847]
Adding to table


PROCESSING GRID CELL ID 2 NO. 2/522 CENTROID (13.49107, 13.52348)
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 61 time steps


In [20]:
count_table

,cell_id,total,lat,lon,datasets,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
0,2,2,13.30037,14.52348,60,165822,173181,177460,177815,174100,179122,173994,177974,139559,130173,128263,129089,147861
1,1,2,13.20037,14.52348,60,257591,258101,255791,255595,257669,257964,257891,258017,256270,253501,253948,252379,255845


In [25]:
count_table.to_csv("output/csv/cell_counts.csv", index=False)

In [ ]:
e_log

In [ ]:
# Visualize Error Cells
e_grid = gpd.GeoDataFrame()
for e in e_log:
    point = Point(e[0], e[1])  # This takes x first and then y
    gdf = gpd.GeoDataFrame(geometry=[point])
    buffer = 0.25
    cell = gpd.GeoDataFrame()
    cell["geometry"] = gdf.buffer(buffer, cap_style="square")
    e_grid = pd.concat([e_grid, cell])
e_grid = e_grid.set_crs("epsg:4326")  # e_grid with same cell size as main grid

# e_grid_aoi = e_grid.dissolve()
# e_grid_fine = create_grid(e_grid_aoi, size) # if require to change the size and make it finer

view_input([e_grid], e_log)

In [ ]:
# Re-run application for cells logged in e_log
if len(e_log) > 0:
    e_log = gen_aoim(e_log, size / 2)